Test of this jupyter notebook

https://github.com/BioSystemsUM/troppo/blob/master/tests/validation/TROPPO%20usage%20and%20benchmark%20notebook.ipynb

In [1]:
import sys;
# sys.path.extend(['E:\\reconstruction', 'E:\\cobamp', 'E:\\reconstruction\\src', 'E:\\cobamp\\src', 'E:/reconstruction'])
import cobra
import framed
import cobamp
import pandas as pd
import numpy as np
import pickle
import scipy as sci
import os
import re

# for testing the algorithms
from cobamp.wrappers import MatFormatReader
from cobamp.wrappers import COBRAModelObjectReader
from troppo.methods.imat import IMAT
from troppo.methods.gimme import GIMME
from troppo.methods.tINIT import tINIT
from troppo.methods.fastcore import FASTcore
from troppo.reconstruction_properties import FastcoreProperties, tINITProperties, GIMMEProperties, IMATProperties
from troppo.utilities.statistics import normalize, z_score

In [2]:
def load_glial_data(path):
    data = pd.read_csv(path, index_col=0)
    normalized_data = normalize(data)
    z = None

    return normalized_data, z

def get_index_core(reactions_ids, data, threshold):
    rx_names = data.index[data['0'] > threshold].tolist()
    index_core = [np.where(reactions_ids == r)[0][0] for r in rx_names]
    return index_core, rx_names

def get_index_values():
    pass

In [3]:
from project import configs
import os
generalmodelmat = os.path.join(configs.datadir,'GeneralModel.mat')

In [4]:
# mat = sci.io.loadmat('./tests/FastCoreTest.mat')['ConsistentRecon2']
model_cobra = cobra.io.load_matlab_model(generalmodelmat)

In [7]:
len(model_cobra.reactions)

10610

In [7]:
mat = sci.io.loadmat(generalmodelmat)['Teff']
#['ConsistentRecon2']

In [18]:
# dir(model_cobra)
# dir(mat)

array([[(array([[array(['AGTim'], dtype='<U5')],
       [array(['AGTix'], dtype='<U5')],
       [array(['ALAR'], dtype='<U4')],
       ...,
       [array(['biomass_reaction_Mphage'], dtype='<U23')],
       [array(['RTOT_3'], dtype='<U6')],
       [array(['ARTFR13'], dtype='<U7')]], dtype=object), <5837x10610 sparse matrix of type '<class 'numpy.float64'>'
	with 40537 stored elements in Compressed Sparse Column format>, array([[    0],
       [    0],
       [-1000],
       ...,
       [    0],
       [    0],
       [    0]], dtype=int16), array([[1000],
       [1000],
       [1000],
       ...,
       [1000],
       [1000],
       [1000]], dtype=uint16), array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=uint8), array([[array(['10fthf5glu[c]'], dtype='<U13')],
       [array(['10fthf5glu[e]'], dtype='<U13')],
       [array(['10fthf5glu[l]'], dtype='<U13')],
       ...,
       [array(['zymst[r]'], dtype='<U8')],
       [array(['zymstnl[c]'], dtype

In [8]:
model = MatFormatReader(mat)
S = model.S
lb, ub = model.get_model_bounds(False, True)
rx_names = model.get_reaction_and_metabolite_ids()[0]

In [14]:
print(len(rx_names))
print(len(model_cobra.reactions))
exp_vector = np.random.random(len(rx_names))
# exp_vector

10610
10610


In [36]:
# print(len(model_cobra.genes))
rx_names

['AGTim',
 'AGTix',
 'ALAR',
 'ARGSL',
 'ARGSS',
 'ASNNm',
 'ASNS1',
 'ASPNATm',
 'ASPTA',
 'ASPTAm',
 'DASPO1p',
 'NACASPAH',
 'r0127',
 'COKECBESr',
 'ACGALK',
 'ACGALK2',
 'ACGAM2E',
 'ACGAM6PSi',
 'ACGAMK',
 'ACGAMPM',
 'ACNAM9PL',
 'ACNAM9PL2',
 'ACNAMPH',
 'ACNMLr',
 'AGDC',
 'AMANK',
 'CHTNASE',
 'CHTNASEe',
 'CMPSAS',
 'CMPSASn',
 'G6PDA',
 'GF6PTA',
 'HEX10',
 'HMR_4124',
 'KDNH',
 'r0013',
 'r0113',
 'r0363',
 'r0364',
 'r0782',
 'r1374',
 'r1375',
 'UAG2EMAi',
 'UAG4E',
 'UAGALDP',
 'UAGDP',
 'HMR_1944',
 'HMR_1958',
 'HMR_1962',
 'HMR_1967',
 'HMR_1968',
 'HMR_1970',
 'HMR_1971',
 'HMR_1976',
 'HMR_1982',
 'HMR_1983',
 'HMR_2014',
 'HMR_2016',
 'HMR_2018',
 'HMR_2029',
 'HMR_2088',
 'HMR_2089',
 'RE1587C',
 'RE1630C',
 'RE1632C',
 'RE2235C',
 'RE2235R',
 'RE2386C',
 'RE3013C',
 'RE3108C',
 'RE3218C',
 'RE3220C',
 'HMR_0942',
 'HMR_0946',
 'HMR_0950',
 'HMR_0954',
 'HMR_0960',
 'HMR_0963',
 'HMR_0979',
 'HMR_0980',
 'HMR_0981',
 'HMR_0985',
 'HMR_0986',
 'HMR_0987',
 'HMR_10

In [16]:
# code for the GIMME test
idx_objective = rx_names.index('biomass_reaction_Mphage')
properties = GIMMEProperties(
    exp_vector=exp_vector,#np.array(gimme_data['0']),
    obj_frac=0.9,
    objectives= [{idx_objective:1}],
    preprocess=True,
    flux_threshold=0.9
)
algorithm = GIMME(S, lb.astype(float), ub.astype(float), properties)
model_GIMME = algorithm.run()

In [17]:
model_GIMME_final = model_cobra.copy() # this is done since it alters the original model_cobra; this way is to guarantee that a new model is changed instead of the original model
r_ids = [r.id for r in model_GIMME_final.reactions]
to_remove_ids = [r_ids[r] for r in np.where(model_GIMME==0)[0]]
model_GIMME_final.remove_reactions(to_remove_ids,True) # this is to get the ids of the reactions to be removed in the model; True is to remove the pending genes/metabolites that with the removal of the reaction can no longer be connected in the network

In [18]:
print('1\'s: ' + str(len(np.where(model_GIMME==1)[0])))
print('2\'s: ' + str(len(np.where(model_GIMME==2)[0])))
print('Genes: ' + str(len(model_GIMME_final.genes)))
print('Metabolites: ' + str(len(model_GIMME_final.metabolites)))
print('Reactions: ' + str(len(model_GIMME_final.reactions)))
print(model_GIMME_final.objective._get_expression())
print(model_GIMME_final.optimize())

1's: 1115
2's: 124
Genes: 658
Metabolites: 1857
Reactions: 1239
1.0*biomass_reaction_Mphage - 1.0*biomass_reaction_Mphage_reverse_6cff5
<Solution 0.000 at 0x1ecd55208>


In [20]:
sum(model_GIMME)

1363.0

In [35]:
csvfilepath = os.path.join(configs.datadir,'merged_Th1.csv')
expressionData = pd.read_csv(csvfilepath)
expressionData.rename(columns={'ENTREZ_GENE_ID':'Gene','Express':'Data'},inplace=True)
expressionData = expressionData.loc[:,['Gene','Data']]
singleGeneNames = expressionData[~expressionData.Gene.str.contains('//')].reset_index(drop=True)
multipleGeneNames = expressionData[expressionData.Gene.str.contains('//')].reset_index(drop=True)
breaksGeneNames = pd.DataFrame(columns=['Gene','Data'])
print(singleGeneNames.shape)
print(multipleGeneNames.shape)
for index,row in multipleGeneNames.iterrows():
    for genename in row['Gene'].split('///'):
        breaksGeneNames = breaksGeneNames.append({'Gene':genename,'Data':row['Data']},ignore_index=True)
print(breaksGeneNames.shape)
GeneExpressions = singleGeneNames.append(breaksGeneNames,ignore_index=True)
print(GeneExpressions.shape)
GeneExpressions.set_index('Gene',inplace=True)
GeneExpressions

(20638, 2)
(1233, 2)
(3429, 2)
(24067, 2)


,Gene,Data
0,1,1
1,10,0
2,100,1
3,1000,0
4,10000,1
5,100009676,0
6,10001,1
7,10002,0
8,10003,0
9,100033411,0


In [106]:
print(len(expressionRxns))
expressionRxns

10610


{'AGTim': 0,
 'AGTix': 0,
 'ALAR': 1,
 'ARGSL': 1,
 'ARGSS': 0,
 'ASNNm': 0,
 'ASNS1': 1,
 'ASPNATm': -1.0,
 'ASPTA': 1,
 'ASPTAm': 1,
 'DASPO1p': 0,
 'NACASPAH': 0,
 'r0127': 0,
 'COKECBESr': 0,
 'ACGALK': -1.0,
 'ACGALK2': -1.0,
 'ACGAM2E': 0,
 'ACGAM6PSi': 1,
 'ACGAMK': 1,
 'ACGAMPM': 0,
 'ACNAM9PL': 1,
 'ACNAM9PL2': 1,
 'ACNAMPH': 0,
 'ACNMLr': -1.0,
 'AGDC': -1,
 'AMANK': 1,
 'CHTNASE': 0,
 'CHTNASEe': 0,
 'CMPSAS': 1,
 'CMPSASn': 1,
 'G6PDA': 1,
 'GF6PTA': 0,
 'HEX10': 0,
 'HMR_4124': 1,
 'KDNH': -1.0,
 'r0013': 1,
 'r0113': 1,
 'r0363': 0,
 'r0364': 0,
 'r0782': 1,
 'r1374': 0,
 'r1375': -1.0,
 'UAG2EMAi': 1,
 'UAG4E': 1,
 'UAGALDP': -1.0,
 'UAGDP': 0,
 'HMR_1944': 0,
 'HMR_1958': 0,
 'HMR_1962': 0,
 'HMR_1967': 0,
 'HMR_1968': 0,
 'HMR_1970': 0,
 'HMR_1971': 0,
 'HMR_1976': 0,
 'HMR_1982': 0,
 'HMR_1983': 0,
 'HMR_2014': 0,
 'HMR_2016': -1,
 'HMR_2018': -1,
 'HMR_2029': -1,
 'HMR_2088': 0,
 'HMR_2089': 0,
 'RE1587C': -1.0,
 'RE1630C': -1.0,
 'RE1632C': -1.0,
 'RE2235C': -1.0,
 

In [105]:
# GeneExpressions.set_index('Gene')
expressionRxns = {}
cnt = 0
for rxn in model_cobra.reactions:
    gene_reaction_rule = rxn.gene_reaction_rule
    gene_ids = re.findall(r'\d+',gene_reaction_rule)
    expressionRxns[rxn.id] = -1.0
    if gene_reaction_rule =='':
        continue
    for id in gene_ids:
        boolval = '-1'
        if id in GeneExpressions.index:
            boolval = '{}'.format(GeneExpressions.at[id,'Data'])
        gene_reaction_rule = gene_reaction_rule.replace('{}'.format(id), boolval, 1)
    
    try:
        gene_reaction_by_rule = float_logical_exp(gene_reaction_rule)
        gene_reaction_by_rule = gene_reaction_by_rule.replace('{','(')
        gene_reaction_by_rule = gene_reaction_by_rule.replace('}',')')
        expressionRxns[rxn.id] = eval(gene_reaction_by_rule)
    except:
        print(gene_reaction_by_rule)
        cnt += 1
print(cnt)

0


In [58]:
for rxn in model_cobra.reactions:
    if len(rxn.gene_reaction_rule)>100:
        print(rxn.gene_reaction_rule)

64816 or 1565 or 1577 or 1562 or 1571 or 1573 or 1588 or 1545 or 1559 or 1558 or 1543 or 1544 or 1580 or 113612 or 1576 or 1551 or 1557 or 29785 or 57834 or 199974 or 66002 or 260293 or 11283 or 1555 or 1572 or 1553 or 1549 or 1548
1543 or 1544 or 1571 or 1576 or 1558 or 1565 or 1548 or 1588 or 1559 or 1580 or 1555 or 1577 or 1549 or 1551 or 1572 or 1562 or 1573 or 11283 or 1545 or 1553 or 199974 or 260293 or 29785 or 64816 or 57834 or 66002
64816 or 1565 or 1577 or 1562 or 1571 or 1573 or 1588 or 1545 or 1559 or 1558 or 1543 or 1544 or 1580 or 113612 or 1576 or 1551 or 1557 or 29785 or 57834 or 199974 or 66002 or 260293 or 11283 or 1555 or 1572 or 1553 or 1549 or 1548
64816 or 1565 or 1577 or 1562 or 1571 or 1573 or 1588 or 1545 or 1559 or 1558 or 1543 or 1544 or 1580 or 113612 or 1576 or 1551 or 1557 or 29785 or 57834 or 199974 or 66002 or 260293 or 11283 or 1555 or 1572 or 1553 or 1549 or 1548
64816 or 1565 or 1577 or 1562 or 1571 or 1573 or 1588 or 1545 or 1559 or 1558 or 1543 or 1

In [59]:
testexp = '(( 2977 ) and ( 2983 )) or 4882 or (( 2982 ) and ( 2974 )) or 2986 or 2984 or 4881 or (( 2977 ) and ( 2974 )) or 3000 or (( 2982 ) and ( 2983 ))'

In [94]:
print(testexp.index('+'))
print(testexp.rindex(')'))

ValueError: substring not found

In [100]:
def float_logical_exp(expressionIn, level=0):
    try:
        loc_r = expressionIn.index(')')
    except:
        if 'and' in expressionIn:
            expressionIn = expressionIn.replace('and', ',')
            expressionIn = 'min{' + expressionIn + '}'
        elif 'or' in expressionIn:
            expressionIn = expressionIn.replace('or', ',')
            expressionIn = 'min{' + expressionIn + '}'
        else:
            expressionIn = expressionIn.replace('[', '')
            expressionIn = expressionIn.replace(']', '')
        return expressionIn
    loc_l = expressionIn[0:loc_r].rindex('(')
    innerstring = expressionIn[loc_l:loc_r+1]
    innerstring = innerstring.replace('(', '[')
    innerstring = innerstring.replace(')', ']')
    if 'and' in innerstring:
        innerstring = innerstring.replace('and', ',')
        innerstring = 'min{'+innerstring+'}'
    elif 'or' in innerstring:
        innerstring = innerstring.replace('or', ',')
        innerstring = 'min{'+innerstring+'}'
    else:
        innerstring = innerstring.replace('[', '')
        innerstring = innerstring.replace(']', '')

    expressionOut = '{}{}{}'.format(expressionIn[0:loc_l], innerstring, expressionIn[loc_r+1:])
    expressionOut = float_logical_exp(expressionOut, level+1)

    return expressionOut


In [96]:
testexp

'(( 2977 ) and ( 2983 )) or 4882 or (( 2982 ) and ( 2974 )) or 2986 or 2984 or 4881 or (( 2977 ) and ( 2974 )) or 3000 or (( 2982 ) and ( 2983 ))'

In [97]:
eval('min([[ 2977 ] , [ 2983 ]])')

[2977]

In [99]:
min([342],[134],500)

TypeError: '<' not supported between instances of 'int' and 'list'